In [1]:
import pandas as pd
from googleapiclient.discovery import build
import re
from datetime import timedelta

In [2]:
mv = pd.read_csv('data/kpop_music_videos.csv')

In [3]:
mv.head()

,Date,Artist,Song Name,Korean Name,Director,Video,Type,Release
0,2020-05-22,Agust D,Daechwita,대취타,NaN,https://youtu.be/qGjAWJ2zWWI,Boy Solo,Major
1,2020-05-21,Yubin,yaya (Me Time),넵넵,NaN,https://youtu.be/BJD0arHF_5c,Girl Solo,Major
2,2020-05-21,OnlyOneOf,Angel (prod. GRAY),NaN,NaN,https://youtu.be/vVTo8p72FxQ,Boy,Major
3,2020-05-20,Ryu Sujeong,Tiger Eyes,NaN,NaN,https://youtu.be/aE6curPGQRY,Girl Solo,Major
4,2020-05-20,Crush,Mayday feat. Joy (Red Velvet),자나깨나,NaN,https://youtu.be/29ycT6fA-Rs,Boy Solo,Major


In [20]:
api_key = 'please use your own key ^^'
youtube = build('youtube', 'v3', developerKey=api_key)

In [21]:
minutes_pattern = re.compile(r'(\d+)M')
seconds_pattern = re.compile(r'(\d+)S')

In [8]:
mv['video_seconds'] = 0
for i in range(len(mv)):
    vid = mv.Video[i][17:]
    request = youtube.videos().list(part = 'contentDetails', id = vid)
    response = request.execute()

    if not response['items']:
        continue
    else:
        duration = response['items'][0]['contentDetails']['duration']
        minutes = minutes_pattern.search(duration)
        seconds = seconds_pattern.search(duration)

        minutes = int(minutes.group(1)) if minutes else 0
        seconds = int(seconds.group(1)) if seconds else 0

        mv.video_seconds[i] = timedelta(minutes=minutes, seconds=seconds).total_seconds()

C:\Users\Alan\AppData\Local\Temp\ipykernel_21436\3849758929.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mv.video_seconds[i] = timedelta(minutes=minutes, seconds=seconds).total_seconds()


In [10]:
mv.video_seconds.value_counts()

0      239
207     65
211     63
215     57
209     56
      ... 
389      1
901      1
131      1
163      1
343      1
Name: video_seconds, Length: 310, dtype: int64

In [22]:
for i in range(len(mv)):
    if mv.video_seconds[i] == 0:
        search_keyword = str(mv.Artist[i]) + ' ' + str(mv['Song Name'][i]) 
        get_id_request = youtube.search().list(q=search_keyword, part='snippet')
        res = get_id_request.execute()
        target_vid = res['items'][0]['id']['videoId']
        
        request = youtube.videos().list(part = 'contentDetails', id = target_vid)
        response = request.execute()

        duration = response['items'][0]['contentDetails']['duration']
        minutes = minutes_pattern.search(duration)
        seconds = seconds_pattern.search(duration)

        minutes = int(minutes.group(1)) if minutes else 0
        seconds = int(seconds.group(1)) if seconds else 0

        mv.video_seconds[i] = timedelta(minutes=minutes, seconds=seconds).total_seconds()

C:\Users\Alan\AppData\Local\Temp\ipykernel_21436\2792871286.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mv.video_seconds[i] = timedelta(minutes=minutes, seconds=seconds).total_seconds()


In [23]:
mv.video_seconds.value_counts()

211     69
207     68
215     63
217     60
213     60
        ..
391      1
1093     1
360      1
102      1
343      1
Name: video_seconds, Length: 314, dtype: int64

In [24]:
mv.to_csv('data/kpop_mv_with_video_seconds.csv', index=False)